In [13]:
# import statements
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as col
from astropy.io import fits
from scipy import ndimage
from photutils import centroid_sources, centroid_com, centroid_1dg, centroid_2dg
from astroCalibReduction import *

In [ ]:
# Outline of reduction script:
# linearity correction ?
# bad pixel correction
# sky subtraction via nodding pairs
# distortion correction ?
# coarse registration, then fine registration via IDL

1. Gather Clio data

In [18]:
root = os.getcwd()
datadir = 'C:/Users/willi/Dropbox (Amherst College)/Research/Follette-Lab/Thesis-data/William/trapezium_data/'

In [63]:
names = []
bands = []
days = []
uniques = []

wavedict = {}
objdict = {}
datedict = {}

images = glob.glob(datadir+'clio_astro/*/*.fit*')
for im in images:
    hdr = fits.getheader(im)
    name = hdr['CID']
    passband = hdr['PASSBAND']
    date = hdr['DATE']
    day = date.split('T')[0]
    unique = day+'&'+passband+'&'+name
    if name not in names:
        names.append(name)
    if passband not in bands:
        bands.append(passband)
    if day not in days:
        days.append(day)
    if unique not in uniques:
        uniques.append(unique)
    wavedict.update({im:passband})
    objdict.update({im:name})
    datedict.update({im:day})

In [61]:
def filelister(filelist, day, band, targ, datedict=datedict, wavedict=wavedict, objdict=objdict):
    thelist = []
    for file in filelist:
        if datedict[file] == day:
            if wavedict[file] == band:
                if objdict[file] == targ:
                    thelist.append(file)
    return thelist

In [66]:
clio_datasets = []
for unique in uniques:
    day, band, name = unique.split('&')
    liste = filelister(images, day, band, name)
    clio_datasets.append(liste)

In [ ]:
def filesorter(filename, foldername, fitskeyword_to_check, keyword, path='./'):
    '''
    The function filesorter takes four arguments: the name of a single FITS file ('filename'), an optional path to the file
    the desired folder name ('foldername'), the desired type of FITS file ('fitskeyword_to_check'), and the header keyword to
    match the FITS file type ('keyword').
    ** This version, modified by William Balmer (Spring 2020), allows for matching of
    RA location up to arcminute accuracy (could be modified to deg or arcsec accuracy) **
    '''
    # If a path to the single FITS file of interest exists, this if-else statement does not do anything. If such a
    # path does not exist, then it will print the file name and let you know that the path does not exist.
    if os.path.exists(path+filename):
        pass
    else:
        print(path+filename + " does not exist or has already been moved.")
        return

    header = fits.getheader(path+filename)
    fits_type = header[keyword]

    # If a parth to the desired folder name exists, this if-else statement does not do anything. If such a path does
    # not exist, then it will let you know it is making a new directory of that folder name and create a directory
    # with the desired file path.
    if os.path.exists(path+foldername):
        pass
    else:
        print("Making new directory: "+path+'/'+foldername)
        os.mkdir(path+'/'+foldername)

    # This if statement first checks if our FITS file is the right type of file we are looking for. If it is, then
    # it designates its location in the desired folder and creates a path for the file. It prints a statement letting
    # you know it is moving the file. It then It then moves the file into the folder by renaming the file from the
    # original "filename" to "foldername/filename"/
    if keyword == 'RASTRNG':
        img_d, img_m, img_s = fits_type.split(':')
        d, m, s = fitskeyword_to_check.split(':')
        location = d+' '+m
        im_loc = img_d+' '+img_m
        if im_loc == location:
            destination = path + foldername + '/'
            print("Moving " + filename + " to: "+ destination + filename)
            os.rename(path+filename, destination + filename)

    elif fits_type == fitskeyword_to_check:
        destination = path + foldername + '/'
        print("Moving " + filename + " to: "+ destination + filename)
        os.rename(path+filename, destination + filename)
    return

In [ ]:
# nod subtraction here

# call H-band data
root = os.getcwd()
root = root + '\\astroCalib'  # only necessary if running from Atom, else comment out
HbandData = glob.glob(root+'\\Trapezium\\Clio_Data\\clio_20191105_06\\trapks*')

# try bad pixel correction
clio_badpix = fits.getdata(root+'\\badpix_fullframe.fit')
# mask bad pixels in images




# begin nod subtraction
H_nod_A = HbandData[0:5] + HbandData[16:20]

H_nod_B = HbandData[5:16]

nod_A_cube = np.zeros((len(H_nod_A), clio_badpix.shape[0], clio_badpix.shape[1]))
nod_B_cube = np.zeros((len(H_nod_B), clio_badpix.shape[0], clio_badpix.shape[1]))

j = 0
k = 0
for i in range(len(HbandData)):
    img = HbandData[i]
    imdata = fits.getdata(img)
    if img in H_nod_A:
        nod_A_cube[j] = badpixelcorrect(imdata, clio_badpix)
        j += 1
    elif img in H_nod_B:
        nod_B_cube[k] = badpixelcorrect(imdata, clio_badpix)
        k += 1
    else:
        break

a_nod_med = np.nanmedian(nod_A_cube, axis=0)
b_nod_med = np.nanmedian(nod_B_cube, axis=0)

badpixd_imdata = nod_A_cube[0]
bg_sub_imdata = badpixd_imdata - b_nod_med

plt.imshow(bg_sub_imdata[290:440, 100:250], origin='lower')

stamps = np.zeros((len(HbandData), 150, 150))

j = 0
k = 0
for i in range(len(HbandData)):
    img = HbandData[i]
    if img in H_nod_A:
        # subtract opposite nod median from single image
        badpixd_imdata = nod_A_cube[j]
        bg_sub_imdata = badpixd_imdata - b_nod_med
        bg_sub_stamp = bg_sub_imdata[180:330, 0:150]
        stamps[i] = bg_sub_stamp
        j += 1
    elif img in H_nod_B:
        badpixd_imdata = nod_B_cube[k]
        bg_sub_imdata = badpixd_imdata - a_nod_med
        bg_sub_stamp = bg_sub_imdata[290:440, 100:250]
        stamps[i] = bg_sub_stamp
        k += 1
    else:
        break

In [ ]:
def crossmed(imcube, cenx=75, ceny=75, box=50):
    im1 = imcube[0]

    xshifts = {}
    yshifts = {}
    cube = np.zeros([imcube.shape[0],im1.shape[0],im1.shape[1]])

    for index in range(imcube.shape[0]):
        im = imcube[index]
        xshifts[index], yshifts[index] = cross_image(im1, im, centerx=cenx, centery=ceny, boxsize=box)
        cube[index,:,:] = shift_image(im,xshifts[index], yshifts[index])

    # Calculate trim edges of new median stacked images so all stacked images of each target have same size
    max_x_shift = int(np.max([xshifts[x] for x in xshifts.keys()]))
    max_y_shift = int(np.max([yshifts[x] for x in yshifts.keys()]))

    print('   Max x-shift={0}, max y-shift={1} (pixels)'.format(max_x_shift,max_y_shift))
    median_image = np.nanmedian(cube, axis=0)

    # Sets the new image boundaries
    # if (max_x_shift > 0) & (max_y_shift > 0): # don't apply cut if no shift!
    #     median_image = median_image[max_x_shift:-max_x_shift,max_y_shift:-max_y_shift]


    print('\n Done stacking!')
    return median_image, cube

In [ ]:
def photcenter(imcube, xinit, yinit):
    '''takes an image cube, finds and shifts to median combine using photutils'''
    im1 = imcube[0]

    xshifts = {}
    yshifts = {}
    cube = np.zeros([imcube.shape[0],im1.shape[0],im1.shape[1]])

    for index in range(imcube.shape[0]):
        im = imcube[index]
        x, y = centroid_sources(data, x_init, y_init, box_size=30,
                                                          centroid_func=centroid_2dg)
        xshifts[index] = x - im1.shape[0]/2.
        yshifts[index] = y - im1.shape[1]/2.
        cube[index,:,:] = shift_image(im,xshifts[index], yshifts[index])

    median_image = np.nanmedian(cube, axis=0)
    return median_image

In [ ]:
med, c = crossmed(stamps, box=340)

test = photcenter(c, 103, 83)


data = c[0]
y_init = (100, 83, 43, 30)
x_init = (68, 103, 85, 85)
x, y = centroid_sources(test, 103, 83, box_size=15,
                        centroid_func=centroid_2dg)
plt.figure(figsize=(8, 4))
plt.imshow(test, origin='lower', interpolation='nearest', cmap='inferno', norm=col.LogNorm(vmin=0.1, vmax=2500))
plt.scatter(x, y, marker='+', s=80, color='red')
plt.tight_layout()